In [42]:
schools

{'abilene-christian': 'Abilene Christian',
 'air-force': 'Air Force',
 'akron': 'Akron',
 'alabama': 'Alabama',
 'alabama-am': 'Alabama A&M',
 'alabama-state': 'Alabama State',
 'albany-ny': 'Albany (NY)',
 'alcorn-state': 'Alcorn State',
 'american': 'American',
 'appalachian-state': 'Appalachian State',
 'arizona': 'Arizona',
 'arizona-state': 'Arizona State',
 'arkansas': 'Arkansas',
 'arkansas-state': 'Arkansas State',
 'arkansas-pine-bluff': 'Arkansas-Pine Bluff',
 'army': 'Army',
 'auburn': 'Auburn',
 'austin-peay': 'Austin Peay',
 'ball-state': 'Ball State',
 'baylor': 'Baylor',
 'bellarmine': 'Bellarmine',
 'belmont': 'Belmont',
 'bethune-cookman': 'Bethune-Cookman',
 'binghamton': 'Binghamton',
 'boise-state': 'Boise State',
 'boston-college': 'Boston College',
 'boston-university': 'Boston University',
 'bowling-green-state': 'Bowling Green State',
 'bradley': 'Bradley',
 'brigham-young': 'Brigham Young',
 'brown': 'Brown',
 'bryant': 'Bryant',
 'bucknell': 'Bucknell',
 'buff

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from time import sleep
import sys

In [4]:

def get_schools(year):
    url = "https://www.sports-reference.com/cbb/seasons/" + str(year) + "-school-stats.html"
    page = urlopen(url).read()
    soup = BeautifulSoup(page)
    count  = 0
    table = soup.find("tbody")
    school_dict = dict()
    for row in table.findAll('td', {"data-stat": "school_name"}):
        school_name = row.getText()
        for a in row.find_all('a', href=True):
            link = a['href'].strip()
            name = link[13:].split("/")[0]
            school_dict[name] = school_name
            
    return school_dict


# In[18]:


def remove_substring_from_dict_values(dictionary, substring):
    for key, value in dictionary.items():
        if type(value) == str:
            dictionary[key] = value.replace(substring, "")
    return dictionary


# In[19]:
school_dat = list()

def scrape_season(school_set, year):
    dfs = []
    final_df=pd.DataFrame()
    for school in school_set:
        print(school + '-' + str(year))
        url = "https://www.sports-reference.com/cbb/schools/" + school + "/" + str(year) + "-schedule.html"
        page = urlopen(url).read()
        soup = BeautifulSoup(page)
        school_dat.append(soup)
        pre_df = dict()
        if (soup.find_all("tbody")[1] != None):
            table = soup.find_all("tbody")[1]
            featuresWanted =  {'opp_name', 'pts', 'opp_pts', 'game_location','game_result','overtimes','wins','losses', 'date_game'} #add more features here!!

            rows = table.findChildren(['tr'])
            for row in rows:
                if (row.find('th', {"scope":"row"}) != None):

                    for f in featuresWanted:
                        cell = row.find("td",{"data-stat": f})

                        a = cell.text.strip().encode()
                        text=a.decode("utf-8")
                        if f in pre_df:
                            pre_df[f].append(text)
                        else:
                            pre_df[f]=[text]
            df = pd.DataFrame.from_dict(pre_df)
            df["opp_name"]= df["opp_name"].apply(lambda row: (row.split("(")[0]).rstrip())
            df["school_name"]=school_set[school]
            df["school_name"] = df["school_name"].apply(lambda row: (row.split("(")[0]).rstrip())
            final_df=pd.concat([final_df,df])
            final_df = final_df[['date_game', 'school_name','game_location','opp_name','pts','opp_pts','overtimes','game_result','wins','losses']]
            final_df['game_location'].replace(['@'], 'Away')
            final_df['game_location'].replace(['N'], 'Neutral')
            final_df['game_location'].replace([''], 'Home')
            final_df['MOV'] = pd.to_numeric(final_df['pts']) - pd.to_numeric(final_df['opp_pts'])
            sleep(4)
    return final_df

In [17]:
season_data = scrape_season(schools, 2023)#fix Nevada

abilene-christian-2023
air-force-2023
akron-2023
alabama-2023
alabama-am-2023
alabama-state-2023
albany-ny-2023
alcorn-state-2023
american-2023
appalachian-state-2023
arizona-2023
arizona-state-2023
arkansas-2023
arkansas-state-2023
arkansas-pine-bluff-2023
army-2023
auburn-2023
austin-peay-2023
ball-state-2023
baylor-2023
arkansas-little-rock-2023
alabama-birmingham-2023


In [13]:
sch_key = iter(schools.keys())

dfs = []
final_df=pd.DataFrame()
for soup in school_dat:
    pre_df = dict()
    if (soup.find_all("tbody")[1] != None):
        table = soup.find_all("tbody")[1]
        featuresWanted =  {'opp_name', 'pts', 'opp_pts', 'game_location','game_result','overtimes','wins','losses', 'date_game'} #add more features here!!

        rows = table.findChildren(['tr'])
        for row in rows:
            if (row.find('th', {"scope":"row"}) != None):

                for f in featuresWanted:
                    cell = row.find("td",{"data-stat": f})

                    a = cell.text.strip().encode()
                    text=a.decode("utf-8")
                    if f in pre_df:
                        pre_df[f].append(text)
                    else:
                        pre_df[f]=[text]
        df = pd.DataFrame.from_dict(pre_df)
        df["opp_name"]= df["opp_name"].apply(lambda row: (row.split("(")[0]).rstrip())
        df["school_name"]=schools[next(sch_key)]
        df["school_name"] = df["school_name"].apply(lambda row: (row.split("(")[0]).rstrip())
        final_df=pd.concat([final_df,df])
        final_df = final_df[['date_game', 'school_name','game_location','opp_name','pts','opp_pts','overtimes','game_result','wins','losses']]
        final_df['game_location'].replace(['@'], 'Away')
        final_df['game_location'].replace(['N'], 'Neutral')
        final_df['game_location'].replace([''], 'Home')
        final_df['MOV'] = pd.to_numeric(final_df['pts']) - pd.to_numeric(final_df['opp_pts'])

In [43]:
import pickle
pickle.dump(pd.concat((season_data,final_df,df)),open('final_df.o','wb'))
pickle.dump(schools,open('schools.o','wb'))

# Analyze Data

In [1]:
import numpy as np
import pandas as pd
import pickle
df = pickle.load(open('final_df.o','rb'))
schools = pickle.load(open('schools.o','rb'))

In [245]:
schools

{'abilene-christian': 'Abilene Christian',
 'air-force': 'Air Force',
 'akron': 'Akron',
 'alabama': 'Alabama',
 'alabama-am': 'Alabama A&M',
 'alabama-state': 'Alabama State',
 'albany-ny': 'Albany (NY)',
 'alcorn-state': 'Alcorn State',
 'american': 'American',
 'appalachian-state': 'Appalachian State',
 'arizona': 'Arizona',
 'arizona-state': 'Arizona State',
 'arkansas': 'Arkansas',
 'arkansas-state': 'Arkansas State',
 'arkansas-pine-bluff': 'Arkansas-Pine Bluff',
 'army': 'Army',
 'auburn': 'Auburn',
 'austin-peay': 'Austin Peay',
 'ball-state': 'Ball State',
 'baylor': 'Baylor',
 'bellarmine': 'Bellarmine',
 'belmont': 'Belmont',
 'bethune-cookman': 'Bethune-Cookman',
 'binghamton': 'Binghamton',
 'boise-state': 'Boise State',
 'boston-college': 'Boston College',
 'boston-university': 'Boston University',
 'bowling-green-state': 'Bowling Green State',
 'bradley': 'Bradley',
 'brigham-young': 'Brigham Young',
 'brown': 'Brown',
 'bryant': 'Bryant',
 'bucknell': 'Bucknell',
 'buff

In [2]:
df = df.reset_index()

In [3]:
Miami_FL_games = df.loc[df['school_name']=='Miami'].iloc[:33]
Miami_FL_games

,index,date_game,school_name,game_location,opp_name,pts,opp_pts,overtimes,game_result,wins,losses,MOV
5874,0,"Mon, Nov 7, 2022",Miami,,Lafayette,67,54,,W,1,0,13.0
5875,1,"Fri, Nov 11, 2022",Miami,,UNC Greensboro,79,65,,W,2,0,14.0
5876,2,"Tue, Nov 15, 2022",Miami,,Florida A&M,87,61,,W,3,0,26.0
5877,3,"Sat, Nov 19, 2022",Miami,N,Providence,74,64,,W,4,0,10.0
5878,4,"Sun, Nov 20, 2022",Miami,N,Maryland,70,88,,L,4,1,-18.0
5879,5,"Wed, Nov 23, 2022",Miami,,St. Francis,79,56,,W,5,1,23.0
5880,6,"Sun, Nov 27, 2022",Miami,@,Central Florida,66,64,,W,6,1,2.0
5881,7,"Wed, Nov 30, 2022",Miami,,Rutgers,68,61,,W,7,1,7.0
5882,8,"Sun, Dec 4, 2022",Miami,@,Louisville,80,53,,W,8,1,27.0
5883,9,"Wed, Dec 7, 2022",Miami,,Cornell,107,105,,W,9,1,2.0


In [4]:
ind_names = np.array(list(schools.values()))
name_to_ind = dict(zip(ind_names,np.arange(len(ind_names))))

In [5]:
df.loc[df['pts']=='','pts'] = 0
df.loc[df['opp_pts']=='','opp_pts'] = 0
df.loc[df['school_name']=='Nevada','MOV'] = df.loc[df['school_name']=='Nevada']['pts'].astype(int)-df.loc[df['school_name']=='Nevada']['opp_pts'].astype(int)

In [13]:
name_dict = {"St. John's":"St. John's (NY)",'Howard Payne':'Howard','Albany':'Albany (NY)',
             'Saint Francis':'Saint Francis (PA)','Queens':'Queens (NC)','St. Francis':'St. Francis (NY)',
             'Purdue Northwest':'Purdue-Fort Wayne','Loyola':'Loyola (IL)',"Saint Mary's":"Saint Mary's (CA)"}
def get_name(n):
    if n in name_dict:
        return name_dict[n]
    else:
        return n

mat = np.zeros((len(ind_names),len(ind_names)))
err = list()
for i in range(len(df)):
    if df.loc[i,'game_result']=='W':
        try:
            
            mat[name_to_ind[get_name(df.loc[i,'opp_name'])],name_to_ind[get_name(df.loc[i,'school_name'])]] += df.loc[i,'MOV']
            #1 + (df.loc[i,'game_location']=='@')
        except Exception as e:
            if df.loc[i,'school_name'] == 'Miami': 
                if any(df.loc[i,'opp_name'] == Miami_FL_games['opp_name']) and any(df.loc[i,'date_game'] == Miami_FL_games['date_game']):
                    mat[name_to_ind[get_name(df.loc[i,'opp_name'])],name_to_ind['Miami (FL)']] += df.loc[i,'MOV']
                    #1 + (df.loc[i,'game_location']=='@')
                else:
                    try:
                        mat[name_to_ind[get_name(df.loc[i,'opp_name'])],name_to_ind['Miami (OH)']] += df.loc[i,'MOV']
                        #1 + (df.loc[i,'game_location']=='@')
                    except:
                        err.append(i)
                    pass
                pass
            elif df.loc[i,'opp_name'] == 'Miami':
                if any(df.loc[i,'school_name']==Miami_FL_games['opp_name']) and any(df.loc[i,'date_game'] == Miami_FL_games['date_game']):
                    mat[name_to_ind['Miami (FL)'],name_to_ind[get_name(df.loc[i,'school_name'])]] += df.loc[i,'MOV']
                    #1 + (df.loc[i,'game_location']=='@')
                else:
                    try:
                        mat[name_to_ind['Miami (OH)'],name_to_ind[get_name(df.loc[i,'school_name'])]] += df.loc[i,'MOV']
                        #1 + (df.loc[i,'game_location']=='@')
                    except:
                        err.append(i)
                    pass
                pass
            err.append(i)
            print(e,i)
            pass

'McMurry' 2
'North American' 7
'Arlington Baptist' 8
'Muskingum' 71
'Concord' 76
'Miami' 83
'Tennessee-Southern' 136
'Fisk' 137
'Rust' 140
'Lane' 177
'Immaculata' 197
'Union' 199
'Virginia Military Institute' 268
'Warren Wilson' 291
'Carlow' 300
'Regent' 302
'Harding' 427
'Lyon' 429
'Bethel' 433
'Champion Christian' 461
"Crowley's Ridge" 467
'Arkansas Baptist' 469
'Ecclesia' 472
'Keystone' 491
'Merchant Marine' 498
'Milligan' 559
'Kentucky Christian' 567
'Lindsey Wilson' 568
'Earlham' 589
'Indiana-South Bend' 592
'Miami' 605
'Arkansas Baptist' 655
'Philander Smith' 663
'Southern Illinois-Edwardsville' 683
'Rhodes' 691
'Campbellsville-Harrodsburg' 722
'Alice Lloyd' 728
'Wabash' 729
'Trevecca Nazarene' 762
'Florida National University' 787
'Trinity Baptist' 788
'Cazenovia' 818
'SUNY-Oneonta' 828
'Eastern Oregon' 857
'Johnson & Wales' 918
'Fairmont State' 959
'Ohio Dominican' 960
'Miami' 966
'Wisconsin-Parkside' 979
'Southern Illinois-Edwardsville' 988
'Westminster' 1020
'Western Oregon' 

In [54]:
df['date_game'] = pd.to_datetime(df['date_game']).astype(int)

In [50]:
from datetime import datetime
datetime.strptime(df.loc[0,'date_game'],"%a, %b %d, %Y")

datetime.datetime(2022, 11, 7, 0, 0)

In [10]:
foodict = {k: v for k, v in name_to_ind.items() if k.startswith('Tex')}
foodict

{'Texas': 304,
 'Texas A&M': 305,
 'Texas A&M-Commerce': 306,
 'Texas A&M-Corpus Christi': 307,
 'Texas Southern': 308,
 'Texas State': 309,
 'Texas Tech': 310,
 'Texas-Rio Grande Valley': 311}

In [427]:
sum(mat[:,172])

272.0

In [438]:
ind_names[258]

"Saint Mary's (CA)"

In [18]:
a = "Nevada"
b = "Arizona State"
pr[name_to_ind[a]],pr[name_to_ind[b]],pr[name_to_ind[a]]>pr[name_to_ind[b]]

(0.003954408579564898, 0.0033387646653695352, True)

In [17]:
import json
with open("weighted_page_rank.json", "w") as outfile:
    json.dump(pr, outfile)

In [14]:
import networkx as nx
G=nx.DiGraph(mat)
pr=nx.pagerank(G,0.4)
#pr

In [12]:
sorted(pr.items(), key=lambda x:x[1],reverse=True)

[(3, 0.007431839162093518),
 (114, 0.006844142604905557),
 (304, 0.006240809513540668),
 (103, 0.006105396215544042),
 (10, 0.005865032638044805),
 (242, 0.005785338906250041),
 (60, 0.00577621871063333),
 (324, 0.005487327628037256),
 (300, 0.005463097641118176),
 (258, 0.005461122551159284),
 (169, 0.005429789177597741),
 (163, 0.0053229013687677885),
 (330, 0.0051952925408369505),
 (133, 0.005124100510897534),
 (74, 0.004922753130397747),
 (123, 0.004778858742367583),
 (24, 0.0047258064880112746),
 (263, 0.004677147765576081),
 (88, 0.0046368467892948795),
 (161, 0.004614649033018435),
 (205, 0.004590925451454976),
 (138, 0.004479612077624424),
 (360, 0.004436533649540386),
 (12, 0.0044266868307786),
 (125, 0.004399322810047846),
 (192, 0.00438079164695855),
 (348, 0.004360937495730329),
 (127, 0.004267658938761508),
 (135, 0.0041882787366533895),
 (19, 0.0041770999299996295),
 (16, 0.004168516717088257),
 (252, 0.004167745154161939),
 (63, 0.00416400823568964),
 (229, 0.00413787908

In [89]:
def pagerank(G, alpha=0.85, personalization=None,
             max_iter=100, tol=1.0e-6, nstart=None, weight='weight',
             dangling=None):
    """Return the PageRank of the nodes in the graph.
  
    PageRank computes a ranking of the nodes in the graph G based on
    the structure of the incoming links. It was originally designed as
    an algorithm to rank web pages.
  
    Parameters
    ----------
    G : graph
      A NetworkX graph.  Undirected graphs will be converted to a directed
      graph with two directed edges for each undirected edge.
  
    alpha : float, optional
      Damping parameter for PageRank, default=0.85.
  
    personalization: dict, optional
      The "personalization vector" consisting of a dictionary with a
      key for every graph node and nonzero personalization value for each node.
      By default, a uniform distribution is used.
  
    max_iter : integer, optional
      Maximum number of iterations in power method eigenvalue solver.
  
    tol : float, optional
      Error tolerance used to check convergence in power method solver.
  
    nstart : dictionary, optional
      Starting value of PageRank iteration for each node.
  
    weight : key, optional
      Edge data key to use as weight.  If None weights are set to 1.
  
    dangling: dict, optional
      The outedges to be assigned to any "dangling" nodes, i.e., nodes without
      any outedges. The dict key is the node the outedge points to and the dict
      value is the weight of that outedge. By default, dangling nodes are given
      outedges according to the personalization vector (uniform if not
      specified). This must be selected to result in an irreducible transition
      matrix (see notes under google_matrix). It may be common to have the
      dangling dict to be the same as the personalization dict.
  
    Returns
    -------
    pagerank : dictionary
       Dictionary of nodes with PageRank as value
  
    Notes
    -----
    The eigenvector calculation is done by the power iteration method
    and has no guarantee of convergence.  The iteration will stop
    after max_iter iterations or an error tolerance of
    number_of_nodes(G)*tol has been reached.
  
    The PageRank algorithm was designed for directed graphs but this
    algorithm does not check if the input graph is directed and will
    execute on undirected graphs by converting each edge in the
    directed graph to two edges.
  
      
    """
    if len(G) == 0:
        return {}
  
    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G
  
    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()
  
    # Choose fixed starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())
  
    if personalization is None:
  
        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise NetworkXError('Personalization dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())
  
    if dangling is None:
  
        # Use personalization vector if dangling vector not specified
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise NetworkXError('Dangling node dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v/s) for k, v in dangling.items())
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
  
    # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:
  
            # this matrix multiply looks odd because it is
            # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]
  
        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return x
    raise NetworkXError('pagerank: power iteration failed to converge '
                        'in %d iterations.' % max_iter)